In [13]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

we will provide some helper functions for you to process web scraped HTML table

In [14]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the List of Falcon 9 and Falcon Heavy launches Wikipage updated on 9th June 2021

In [15]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

### TASK 1: Request the Falcon9 Launch Wiki page from its URL
First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.

In [16]:
response = requests.get(static_url)
if response.status_code != 200:
    print(f"Error fetching the page: {response.status_code}")
    sys.exit(1)

Create a BeautifulSoup object from the HTML response

In [17]:
soup = BeautifulSoup(response.text, 'html.parser')

Print the page title to verify if the BeautifulSoup object was created properly

In [18]:
# print the title of the page
print(soup.title.string)

List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header
Next, we want to collect all relevant column names from the HTML table header

Let's try to find all tables on the wiki page first. If you need to refresh your memory about BeautifulSoup, please check the external reference link towards the end of this lab

In [19]:
# find all tables in the HTML, then assign the result to a list named 'html tables'
html_tables = soup.find_all('table', {'class': 'wikitable'})

starting from the third table is our target table contains the actual launch records.

In [20]:
# print the third table, assign the result to a variable named 'first launch_table'
first_launch_table = html_tables[2]
# print the first launch table
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a><sup class="reference" id="cite_ref-booster_11-2"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-2"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9

Next, we just need to iterate through the <th> elements and apply the provided extract_column_from_header() to extract column name one by one

In [21]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

In [22]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


### TASK 3: Create a data frame by parsing the launch HTML tables
We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe

In [23]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the launch_dict with launch records extracted from table rows.

Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links B0004.1[8], missing values N/A [e], inconsistent formatting, etc.

To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the launch_dict. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:

In [28]:
def extract_text(cell):
    """Return the text of the first <a> in the cell, or the cell’s own text."""
    link = cell.find('a')
    if link and link.string:
        return link.string.strip()
    else:
        return cell.get_text(strip=True)


launches = []
extracted_row = 0

for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        # --- determine whether this row has a numeric flight_number in its <th> ---
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False

        # gather all the <td> cells
        cells = rows.find_all('td')

        # only proceed if this row is a numbered launch and has at least 9 cells
        if flag and len(cells) >= 9:
            extracted_row += 1

            # parse date & time
            datatimelist = date_time(cells[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]

            # build the dict
            launch_dict = {
                "Flight No.": flight_number,
                "Date": date,
                "Time": time,
                "Version Booster": booster_version(cells[1]) or extract_text(cells[1]),
                "Launch Site": extract_text(cells[2]),
                "Payload": extract_text(cells[3]),
                "Payload mass": get_mass(cells[4]),
                "Orbit": extract_text(cells[5]),
                "Customer": extract_text(cells[6]),
                "Launch outcome": list(cells[7].strings)[0].strip(),
                "Booster landing": landing_status(cells[8]),
            }

            launches.append(launch_dict)

# at the end, you have:
print(f"Extracted {extracted_row} launches")
# and your parsed data is in `launches`

df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })


Extracted 121 launches


In [30]:
df = pd.DataFrame(launches)

# quickly inspect
print(df.head())

  Flight No.             Date   Time   Version Booster Launch Site  \
0          1      4 June 2010  18:45  F9 v1.07B0003.18       CCAFS   
1          2  8 December 2010  15:43  F9 v1.07B0004.18       CCAFS   
2          3      22 May 2012  07:44  F9 v1.07B0005.18       CCAFS   
3          4   8 October 2012  00:35  F9 v1.07B0006.18       CCAFS   
4          5     1 March 2013  15:10  F9 v1.07B0007.18       CCAFS   

                                Payload Payload mass Orbit Customer  \
0  Dragon Spacecraft Qualification Unit            0   LEO   SpaceX   
1                                Dragon            0   LEO     NASA   
2                                Dragon       525 kg   LEO     NASA   
3                          SpaceX CRS-1     4,700 kg   LEO     NASA   
4                          SpaceX CRS-2     4,877 kg   LEO     NASA   

  Launch outcome Booster landing  
0        Success         Failure  
1        Success         Failure  
2        Success    No attempt\n  
3        Suc

In [31]:
# if you want to persist it to CSV:
df.to_csv('spacex_launches.csv', index=False)
